In [1]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 115.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [2]:
from bertopic import BERTopic 
import json
import pandas as pd 
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd 

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ecommerce_data.csv')

In [4]:
df

,Text,label
0,Urban Ladder Eisner Low Back Study-Office Comp...,Household
1,"Contrast living Wooden Decorative Box,Painted ...",Household
2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics
3,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing & Accessories
4,Indira Designer Women's Art Mysore Silk Saree ...,Clothing & Accessories
...,...,...
23995,Marvel Physics MCQ's for MHT - CET,Books
23996,Internet Download Manager | Lifetime License |...,Books
23997,Sadhubela's Handcrafted Iron Degchi Handi Pot ...,Household
23998,Audio-Technica AT-LP60 Automatic Belt Driven D...,Electronics


In [5]:
df1 = df[df['Text'].str.len() > 30].reset_index()
df1

,index,Text,label
0,0,Urban Ladder Eisner Low Back Study-Office Comp...,Household
1,1,"Contrast living Wooden Decorative Box,Painted ...",Household
2,2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics
3,3,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing & Accessories
4,4,Indira Designer Women's Art Mysore Silk Saree ...,Clothing & Accessories
...,...,...,...
23422,23995,Marvel Physics MCQ's for MHT - CET,Books
23423,23996,Internet Download Manager | Lifetime License |...,Books
23424,23997,Sadhubela's Handcrafted Iron Degchi Handi Pot ...,Household
23425,23998,Audio-Technica AT-LP60 Automatic Belt Driven D...,Electronics


In [6]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer


# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=10, n_components=2, min_dist=0.01, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=1, metric='euclidean', cluster_selection_method='leaf', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

nr_topics = 50


# deal with df if needed
if type(df1['Text']) is list:
    text = df1['Text']
else:
    text = df1['Text'].tolist()



model = BERTopic(
    nr_topics=nr_topics,
    language='english', calculate_probabilities=True,
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality to 2D or 3D
    hdbscan_model=hdbscan_model,              # Step 3 - Identify Clusters in reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic represenations
    verbose=True
)
topics, probs = model.fit_transform(text)

Batches:   0%|          | 0/733 [00:00<?, ?it/s]

2023-05-18 14:18:13,447 - BERTopic - Transformed documents to Embeddings
2023-05-18 14:18:54,419 - BERTopic - Reduced dimensionality
2023-05-18 14:25:27,544 - BERTopic - Clustered reduced embeddings
2023-05-18 14:26:56,115 - BERTopic - Reduced number of topics from 543 to 50


In [7]:
import pickle

In [ ]:
pickle.dump(model, open('/content/model_50_topics', 'wb'))

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
model.get_topic_info()

,Topic,Count,Name
0,-1,3390,-1_men thermals_thermals_wear_fabric
1,0,4891,0_cotton_fabric_lingerie_socks
2,1,1700,1_bluetooth speaker_bluetooth_jbl_wireless
3,2,1546,2_usb_usb port_flash drive_sandisk
4,3,1500,3_literature_novel_bestselling author_book
5,4,1438,4_furniture_dining table_chairs_seating
6,5,1126,5_hindu_sikh_writer_literature
7,6,996,6_hdmi cable_hdmi_1080p_blu ray
8,7,882,7_tealight candles_light candle_tea light_teal...
9,8,779,8_thermal vest_baby thermal_thermal set_thermals


In [9]:
model.visualize_topics()

In [10]:
model.visualize_hierarchy()

In [11]:
model.visualize_barchart(top_n_topics = 20, n_words=10)

In [12]:
generated_labels = model.generate_topic_labels()
generated_labels

['-1_men thermals_thermals_wear',
 '0_cotton_fabric_lingerie',
 '1_bluetooth speaker_bluetooth_jbl',
 '2_usb_usb port_flash drive',
 '3_literature_novel_bestselling author',
 '4_furniture_dining table_chairs',
 '5_hindu_sikh_writer',
 '6_hdmi cable_hdmi_1080p',
 '7_tealight candles_light candle_tea light',
 '8_thermal vest_baby thermal_thermal set',
 '9_designed exam_exam_professionals arihant',
 '10_decals_decal applied_wall stickers',
 '11_bean bags_bean bag_comfy bean',
 '12_wear kurta_cotton kurta_silk kurta',
 '13_blazer tuxedo_favoroski men_tuxedo',
 '14_laptop adapter_adapter charger_power adapter',
 '15_inner engineering_engineering_engineer',
 '16_inkjet printer_printer hp_deskjet ink',
 '17_lens cleaning_cleaning kit_microfiber cleaning',
 '18_cabinet knobs_door knobs_knobs glazed',
 '19_air purifier_hepa filter_air quality',
 '20_powers subconscious_power subconscious_subconscious mind',
 '21_curtains balcony_balcony windows_curtain verandas',
 '22_karl marx_marx engels_marx

In [13]:
df2 = pd.DataFrame({"topic": topics, "document": df1.Text, "original label": df1.label })
df2

,topic,document,original label
0,4,Urban Ladder Eisner Low Back Study-Office Comp...,Household
1,22,"Contrast living Wooden Decorative Box,Painted ...",Household
2,2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics
3,0,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing & Accessories
4,0,Indira Designer Women's Art Mysore Silk Saree ...,Clothing & Accessories
...,...,...,...
23422,29,Marvel Physics MCQ's for MHT - CET,Books
23423,33,Internet Download Manager | Lifetime License |...,Books
23424,10,Sadhubela's Handcrafted Iron Degchi Handi Pot ...,Household
23425,1,Audio-Technica AT-LP60 Automatic Belt Driven D...,Electronics


In [14]:
numbers = [int(item.split('_')[0]) for item in generated_labels]
text = [item.split('_')[1:] for item in generated_labels]

# Creating the DataFrame
df3 = pd.DataFrame({'topic': numbers, 'Topic_words': text}).reset_index(drop =True)
df3

,topic,Topic_words
0,-1,"[men thermals, thermals, wear]"
1,0,"[cotton, fabric, lingerie]"
2,1,"[bluetooth speaker, bluetooth, jbl]"
3,2,"[usb, usb port, flash drive]"
4,3,"[literature, novel, bestselling author]"
5,4,"[furniture, dining table, chairs]"
6,5,"[hindu, sikh, writer]"
7,6,"[hdmi cable, hdmi, 1080p]"
8,7,"[tealight candles, light candle, tea light]"
9,8,"[thermal vest, baby thermal, thermal set]"


In [15]:
df4 = df2.merge(df3, on='topic', how='left').reset_index(drop =True)
df4

,topic,document,original label,Topic_words
0,4,Urban Ladder Eisner Low Back Study-Office Comp...,Household,"[furniture, dining table, chairs]"
1,22,"Contrast living Wooden Decorative Box,Painted ...",Household,"[karl marx, marx engels, marx]"
2,2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics,"[usb, usb port, flash drive]"
3,0,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing & Accessories,"[cotton, fabric, lingerie]"
4,0,Indira Designer Women's Art Mysore Silk Saree ...,Clothing & Accessories,"[cotton, fabric, lingerie]"
...,...,...,...,...
23422,29,Marvel Physics MCQ's for MHT - CET,Books,"[gate questions, mathematics gate, solutions g..."
23423,33,Internet Download Manager | Lifetime License |...,Books,"[mcafee, virus protection, download mcafee]"
23424,10,Sadhubela's Handcrafted Iron Degchi Handi Pot ...,Household,"[decals, decal applied, wall stickers]"
23425,1,Audio-Technica AT-LP60 Automatic Belt Driven D...,Electronics,"[bluetooth speaker, bluetooth, jbl]"


In [16]:
model.get_representative_docs(1)

['JBL GO Portable Wireless Bluetooth Speaker with Mic (Gray) Colour:Grey   Product Description Packed in a pocket-size body, JBL Go speaker delivers powerful and high impact sound. It takes the entertainment quotient higher with its effective audio output which comes at a reasonable price range. This speaker is designed with minimalistic approach to size and is easily portable. You can carry it to family picnics, beach parties, sports practice or to your evening walks for keeping yourself entertained. The beauty of this JBL speaker is that it comes with Bluetooth connectivity. You can play your favourite sound tracks available on your mobile phone by simply pairing this speaker to it. It gives you 5 hours of non-stop entertainment once fully charged. This JBL speaker also has Aux-In slot available that enables you to connect various other media devices to it. The built-in speakerphone available on this JBL speaker has noise cancellation technology. It allows you to take calls convenien

In [19]:

def get_docs():
  top_3_docs = []
  for topic in range(0,49):
    top_3_docs.append(model.get_representative_docs(topic))
  return top_3_docs


